# **Human Action Recognition with MMPose and Spatio-Tempoal Graph Convolutional Network**
HigherHRNet48 + STGCN

In [ ]:
%%capture
!pip install wandb -qqq
import wandb

In [ ]:
%%writefile /root/.netrc
machine api.wandb.ai
  login user
  password 35238dfceafeb9e2e0695631c642d4b3913b0905

Writing /root/.netrc


In [ ]:
!wandb login

wandb: Currently logged in as: wchoi (use `wandb login --relogin` to force relogin)



# Download the Florence 3D action dataset.

## Florence 3D actions dataset

The dataset collected at the University of Florence during 2012, has been captured using a Kinect camera. It includes 9 activities: wave, drink from a bottle, answer phone,clap, tight lace, sit down, stand up, read watch, bow. During acquisition, 10 subjects were asked to perform the above
actions for 2/3 times. This resulted in a total of 215 activity samples.
We suggest a leave-one-actor-out protocol: train your classifier using all the sequences from 9 out of 10 actors and test on the remaining one. Repeat this procedure for all actors and average the 10 classification accuracy values.

Actions 
1.	wave
2.	drink from a bottle
3.	answer phone
4.	clap
5.	tight lace
6.	sit down
7.	stand up
8.	read watch
9.	bow

Videos depicting the actions are named GestureRecording_Id\<ID_GESTURE\>actor\<ID_ACTOR\>idAction\<ID_ACTION\>category\<ID_CATEGORY\>.avi
The file The file Florence_dataset_Features.txt contains all the pose features with annotate actor and actions. Each line is formatted according to the following:

%idvideo idactor idcategory  f1....fn

where f1-f24 are our normalized body part coordinates
and f25 is the normalized frame value.

Specifically:  
  elbows: f1-f6; (1-3 left elbow, 4-6 right elbow, same applies for all other joints)  
  wrists: f13-f18  
  knees: f7-f12  
  ankles: f19-f24  
  normalized frame value: f25  

The file Florence_dataset_WorldCoordinates.txt
Contains the world coordinates for all the joints. Thanks to Maxime Devanne for parsing this data! Each line is formatted according to the following:

%idvideo idactor idcategory  f1....fn
where f1-f45 are world coordinates of all the 15 joints.

Specifically:  
  Head: f1-f3  
  Neck: f4-f6  
  Spine: f7-f9  
  Left Shoulder: f10-f12  
  Left Elbow: f13-f15  
  Left Wrist: f16-f18  
  Right Shoulder: f19-f21  
  Right Elbow: f22-f24  
  Right Wrist: f25-f27  
  Left Hip: f28-f30  
  Left Knee: f31-f33  
  Left Ankle: f34-f36  
  Right Hip: f37-f39  
  Right Knee: f40-f42  
  Right Ankle: f43-f45  


In [ ]:
%%shell
curl https://www.micc.unifi.it/vim/wp-content/uploads/datasets/florence3d_actions.zip -o /content/florence3d_actions.zip
unzip -o -q florence3d_actions.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  303M  100  303M    0     0  11.2M      0  0:00:27  0:00:27 --:--:-- 6332k


## CCAM actions dataset

In [ ]:
%cd /content

# video download
file_id = "1wUFcGp8sXcoiGSRmxL3a4sRw13I56Hk1"
file_name = "ccam_actions.zip"
!wget --load-cookies ~/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies ~/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=$file_id' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=$file_id" -O $file_name && rm -rf ~/cookies.txt
!unzip -o -q ccam_actions.zip

# pose deta
file_id, file_name = "1_0fReh6XdE4Y0Q1-Wg1Mj1lrtTW5MPi-", "dataset_florence.zip"
!wget 'https://docs.google.com/uc?export=download&id=$file_id' -O $file_name
file_id, file_name = "1zUFdwiog47w88GKZAisJFg8GqD0nb8vD", "dataset_ccam.zip"
!wget 'https://docs.google.com/uc?export=download&id=$file_id' -O $file_name

# front views
!mkdir ccam_actions_org
file_list = [
             ("WaveHand_cam_rosberry1_color_image_raw.mp4","1FIenejZENb9FrnxGUFjG4kI-ANmLokFu"),
             ("TightLace_cam_rosberry1_color_image_raw.mp4", "1WtqrJ2eK88jqn8ryfmSBroHHE51Qvg4X"),
             ("Stationary_cam_rosberry1_color_image_raw.mp4", "1BE1BvQOmM6-PAn8H3m4_Hv1pN8cEB-nT"),
             ("SitDownStandUp_cam_rosberry1_color_image_raw.mp4", "1nsR3VRgK_8HPLCb99149KvJih1ctZR96"),
             ("ReadWatch_cam_rosberry1_color_image_raw.mp4", "1p4kUOaJtDxrruWk7cBULQ9zEh0ndPJb8"),
             ("G230_Master_cam_rosberry1_color_image_raw.mp4", "1Kj7aUfFe5ufg9tqMUcC_j0oHSx7EHLkn"),
             ("DrinkFromBottle_cam_rosberry1_color_image_raw.mp4", "1iQlLiHjT2ewN4piBiAvGLcTwVTe1Dhtz"),
             ("Clapping_cam_rosberry1_color_image_raw.mp4", "1jkJmIAV9SfcqRPY45BEP5cb6rOac5j3f"),
             ("Bow_cam_rosberry1_color_image_raw.mp4", "1LKVDZ_FORByXGRGSp7GgSMsd_Q4ZrEWZ"),
             ("AnswerPhone_cam_rosberry1_color_image_raw.mp4","1FQhu9bRJqe7PGNniktk6TUaXGn0PjFRd"),
            ]

for file_name, file_id in file_list:
  print(file_id, file_name)
  file_name = "ccam_actions_org/"+file_name
  !wget --load-cookies ~/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies ~/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=$file_id' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=$file_id" -O $file_name && rm -rf ~/cookies.txt


/content
--2021-02-17 20:10:38--  https://docs.google.com/uc?export=download&confirm=b991&id=1wUFcGp8sXcoiGSRmxL3a4sRw13I56Hk1
Resolving docs.google.com (docs.google.com)... 74.125.195.100, 74.125.195.138, 74.125.195.101, ...
Connecting to docs.google.com (docs.google.com)|74.125.195.100|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0c-58-docs.googleusercontent.com/docs/securesc/1h8r7vftoaf5eh58bpfmoqag5ofh4h7u/ap99iqpis7qt371n2t9q8058p12mapgn/1613592600000/13244527368295625576/13098616332107508270Z/1wUFcGp8sXcoiGSRmxL3a4sRw13I56Hk1?e=download [following]
--2021-02-17 20:10:39--  https://doc-0c-58-docs.googleusercontent.com/docs/securesc/1h8r7vftoaf5eh58bpfmoqag5ofh4h7u/ap99iqpis7qt371n2t9q8058p12mapgn/1613592600000/13244527368295625576/13098616332107508270Z/1wUFcGp8sXcoiGSRmxL3a4sRw13I56Hk1?e=download
Resolving doc-0c-58-docs.googleusercontent.com (doc-0c-58-docs.googleusercontent.com)... 74.125.20.132, 2607:f8b0:400e:c07::84
Co

# Functions

In [ ]:
import cv2
from tqdm.notebook import trange, tqdm

def frame_iter(capture, description = ""):
  def _iterator():
    while capture.grab():
      yield capture.retrieve()

  return tqdm(
    _iterator(),
    desc=description,
    total=int(capture.get(cv2.CAP_PROP_FRAME_COUNT)),
    leave=False,
  )


def process_mmdet_results(mmdet_results, cat_id=0):
  """Process mmdet results, and return a list of bboxes.

  :param mmdet_results:
  :param cat_id: category id (default: 0 for human)
  :return: a list of detected bounding boxes
  """
  if isinstance(mmdet_results, tuple):
      det_results = mmdet_results[0]
  else:
      det_results = mmdet_results
  return det_results[cat_id]


# visualization
def show_local_mp4_video(file_name, width=640, height=480):
  import io
  import base64
  from IPython.display import HTML
  video_encoded = base64.b64encode(io.open(file_name, 'rb').read())
  return HTML(data='''<video width="{0}" height="{1}" alt="test" controls>
                        <source src="data:video/mp4;base64,{2}" type="video/mp4" />
                      </video>'''.format(width, height, video_encoded.decode('ascii')))

# Installation

## Install MMPose

In [ ]:
%%shell
pip install -q tqdm lazy-import
# This installation takes a long time due to the compiation of mmcv-full
pip install mmcv-full -f https://download.openmmlab.com/mmcv/dist/cu101/torch1.7.0/index.html
pip install mmdet

Looking in links: https://download.openmmlab.com/mmcv/dist/cu101/torch1.7.0/index.html
     |████████████████████████████████| 24.0MB 122kB/s 
     |████████████████████████████████| 194kB 8.6MB/s 
     |████████████████████████████████| 542kB 9.0MB/s 
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15358 sha256=a01464185aa07bf2a4859154cb792cec1cdc821faa153065e1604d10abc78b7d
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
  Created wheel for mmpycocotools: filename=mmpycocotools-12.0.3-cp36-cp36m-linux_x86_64.whl size=265909 sha256=49d7facd9701236a6be1c970d2637f8fb9a724aab363481ee5e237e390146efd
  Stored in directory: /root/.cache/pip/wheels/a2/b0/8d/3307912785a42bc80f673946fac676d5c596eee537af7a599c
Successfully built terminaltables mmpycocotools


In [ ]:
%%shell
cd /content

git clone https://github.com/open-mmlab/mmpose.git
cd mmpose
pip install -q -r requirements.txt
python setup.py -q develop

Cloning into 'mmpose'...
remote: Enumerating objects: 192, done.
remote: Counting objects: 100% (192/192), done.
remote: Compressing objects: 100% (158/158), done.
remote: Total 6398 (delta 70), reused 103 (delta 34), pack-reused 6206
Receiving objects: 100% (6398/6398), 16.29 MiB | 20.29 MiB/s, done.
Resolving deltas: 100% (4120/4120), done.
     |████████████████████████████████| 51kB 4.3MB/s 
     |████████████████████████████████| 317kB 12.3MB/s 
     |████████████████████████████████| 81kB 9.0MB/s 
     |████████████████████████████████| 51kB 7.3MB/s 
     |████████████████████████████████| 112kB 11.6MB/s 
     |████████████████████████████████| 81kB 9.3MB/s 
     |████████████████████████████████| 14.5MB 253kB/s 
     |████████████████████████████████| 4.1MB 50.2MB/s 
     |████████████████████████████████| 71kB 8.8MB/s 
     |████████████████████████████████| 51kB 9.1MB/s 
     |████████████████████████████████| 51kB 8.2MB/s 
     |████████████████████████████████| 952kB 46.0MB/

## Install MMSkeleton

In [ ]:
%%shell
cd /content
git clone https://github.com/open-mmlab/mmdetection.git

Cloning into 'mmdetection'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 15707 (delta 4), reused 0 (delta 0), pack-reused 15688
Receiving objects: 100% (15707/15707), 16.89 MiB | 19.20 MiB/s, done.
Resolving deltas: 100% (10784/10784), done.


In [ ]:
%%shell
cd /content
git clone https://github.com/open-mmlab/mmskeleton.git
cd mmskeleton
python setup.py develop

cd mmskeleton/ops/nms/
python setup_linux.py develop
cd ../../../


Cloning into 'mmskeleton'...
remote: Enumerating objects: 3065, done.
remote: Total 3065 (delta 0), reused 0 (delta 0), pack-reused 3065
Receiving objects: 100% (3065/3065), 93.35 MiB | 35.91 MiB/s, done.
Resolving deltas: 100% (1596/1596), done.
Compiling mmskeleton/ops/nms/cpu_nms.pyx because it changed.
[1/1] Cythonizing mmskeleton/ops/nms/cpu_nms.pyx
/usr/local/lib/python3.6/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/mmskeleton/mmskeleton/ops/nms/cpu_nms.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
/usr/local/lib/python3.6/dist-packages/setuptools/dist.py:461: UserWarning: Normalizing '0.7.rc1+b4c076b' to '0.7rc1+b4c076b'
  warnings.warn(tmpl.format(**locals()))
running develop
running egg_info
creating mmskeleton.egg-info
writing mmskeleton.egg-info/PKG-INFO
writing dependency_links to mmskeleton.egg-info/dependency_links.txt
writing requi

## File change and generation

In [ ]:
%cd /content/mmskeleton/

/content/mmskeleton


### /content/mmskeleton/mmskl.py

In [ ]:
%%writefile /content/mmskeleton/mmskl.py
import argparse
import os
import sys
import logging

import torch
import mmskeleton
from mmcv import Config
from mmskeleton.utils import call_obj, set_attr, get_attr
""" Configuration Structure

argparse_cfg:
  <shortcut_name 1>:
    bind_to: <full variable path>
    help: <help message>
  <shortcut_name 2>:
    ...

processor_cfg:
  name: <full processor path>
  ...

"""

config_shortcut = dict(
    pose_demo_HD='./configs/pose_estimation/pose_demo_HD.yaml',
    pose_demo='./configs/pose_estimation/pose_demo.yaml')


def parse_cfg():

    parser = argparse.ArgumentParser(description='Run a processor.')
    parser.add_argument('config', help='configuration file path')

    if len(sys.argv) <= 1:
        args = parser.parse_args()
        return
    if sys.argv[1] == '-h' or sys.argv[1] == '--help':
        args = parser.parse_args()
        return

    # pop positional args
    config_args = []
    tmp = []
    for i, arg in enumerate(sys.argv):
        if i > 1:
            if arg[0] != '-':
                tmp.append(i)
            else:
                break

    for i in tmp[::-1]:
        config_args.append(sys.argv.pop(i))
    branch = config_args

    # load argument setting from configuration file
    if sys.argv[1] in config_shortcut:
        sys.argv[1] = config_shortcut[sys.argv[1]]

    print('Load configuration information from {}'.format(sys.argv[1]))
    cfg = Config.fromfile(sys.argv[1])
    for b in branch:
        if b in cfg:
            cfg = get_attr(cfg, b)
        else:
            print('The branch "{}" can not be found in {}.'.format(
                '-'.join(branch), sys.argv[1]))
            return dict()
    if 'description' in cfg:
        parser.description = cfg.description
    if 'argparse_cfg' not in cfg:
        cfg.argparse_cfg = dict()
    for key, info in cfg.argparse_cfg.items():
        if 'bind_to' not in info:
            continue
        default = get_attr(cfg, info['bind_to'])
        if 'type' not in info:
            if default is not None:
                info['type'] = type(default)
        else:
            info['type'] = eval(info['type'])
        kwargs = dict(default=default)
        kwargs.update({k: v for k, v in info.items() if k != 'bind_to'})
        parser.add_argument('--' + key, **kwargs)
    args = parser.parse_args()

    # update config from command line
    for key, info in cfg.argparse_cfg.items():
        if 'bind_to' not in info:
            continue
        value = getattr(args, key)
        set_attr(cfg, info['bind_to'], value)

    # replace pre_defined arguments in configuration files
    def replace(cfg, **format_args):
        if isinstance(cfg, str):
            return cfg.format(**format_args)
        if isinstance(cfg, dict):
            for k, v in cfg.items():
                set_attr(cfg, k, replace(v, **format_args))
        elif isinstance(cfg, list):
            for k in range(len(cfg)):
                cfg[k] = replace(cfg[k], **format_args)
        return cfg

    format_args = dict()
    format_args['config_path'] = args.config
    format_args['config_name'] = os.path.basename(format_args['config_path'])
    format_args['config_prefix'] = format_args['config_name'].split('.')[0]
    cfg = replace(cfg, **format_args)
    return cfg


def main():
    cfg = parse_cfg()
    if 'processor_cfg' in cfg:
        call_obj(**cfg.processor_cfg)
    else:
        print('No processor specified.')


if __name__ == "__main__":
    torch.multiprocessing.set_start_method('spawn')
    main()


Overwriting /content/mmskeleton/mmskl.py


### /content/mmskeleton/mmskeleton/processor/recognition.py

In [ ]:
%%writefile mmskeleton/processor/recognition.py
from collections import OrderedDict
import torch
import logging
import numpy as np
from mmskeleton.utils import call_obj, import_obj, load_checkpoint
from mmcv.runner import EpochBasedRunner
from mmcv import Config, ProgressBar
from mmcv.parallel import MMDataParallel
from mmdet.utils import get_root_logger

import wandb

def test(model_cfg,
         dataset_cfg,
         checkpoint,
         batch_size=None,
         gpu_batch_size=None,
         gpus=-1,
         workers=4):

    # calculate batch size
    if gpus < 0:
        gpus = torch.cuda.device_count()
    if (batch_size is None) and (gpu_batch_size is not None):
        batch_size = gpu_batch_size * gpus
    assert batch_size is not None, 'Please appoint batch_size or gpu_batch_size.'

    dataset = call_obj(**dataset_cfg)
    data_loader = torch.utils.data.DataLoader(dataset=dataset,
                                              batch_size=batch_size,
                                              shuffle=False,
                                              num_workers=workers)

    # put model on gpus
    if isinstance(model_cfg, list):
        model = [call_obj(**c) for c in model_cfg]
        model = torch.nn.Sequential(*model)
    else:
        model = call_obj(**model_cfg)
    load_checkpoint(model, checkpoint, map_location='cpu')
    model = MMDataParallel(model, device_ids=range(gpus)).cuda()
    model.eval()

    results = []
    labels = []
    prog_bar = ProgressBar(len(dataset))
    for data, label in data_loader:
        with torch.no_grad():
            output = model(data)
            output = model(data).data.cpu().numpy()

        results.append(output)
        labels.append(label)
        for i in range(len(data)):
            prog_bar.update()
    results = np.concatenate(results)
    labels = np.concatenate(labels)
    
    print()
    print(np.argmax(results, axis=1))
    print(labels)

    print('Top 1: {:.2f}%'.format(100 * topk_accuracy(results, labels, 1)))
    print('Top 3: {:.2f}%'.format(100 * topk_accuracy(results, labels, 3)))


def train(
        work_dir,
        model_cfg,
        loss_cfg,
        dataset_cfg,
        optimizer_cfg,
        total_epochs,
        training_hooks,
        batch_size=None,
        gpu_batch_size=None,
        workflow=[('train', 1)],
        gpus=-1,
        log_level=0,
        workers=4,
        resume_from=None,
        load_from=None,
):
    global loss
    logger = get_root_logger(log_level=log_level)
    config = {
      'workdir': work_dir,
      'model_cfg': model_cfg,
      'loss_cfg': loss_cfg,
      'dataset_cfg': dataset_cfg,
      'optimizer_cfg': optimizer_cfg,
      'total_epochs': total_epochs,
      'training_hooks': training_hooks,
    }

    with wandb.init(project="STGCN", config=config):
      # calculate batch size
      if gpus < 0:
          gpus = torch.cuda.device_count()
      if (batch_size is None) and (gpu_batch_size is not None):
          batch_size = gpu_batch_size * gpus
      assert batch_size is not None, 'Please appoint batch_size or gpu_batch_size.'

      # prepare data loaders
      if isinstance(dataset_cfg, dict):
          dataset_cfg = [dataset_cfg]
      data_loaders = [
          torch.utils.data.DataLoader(dataset=call_obj(**d),
                                      batch_size=batch_size,
                                      shuffle=True,
                                      num_workers=workers,
                                      drop_last=True) for d in dataset_cfg
      ]

    
      # put model on gpus
      if isinstance(model_cfg, list):
          model = [call_obj(**c) for c in model_cfg]
          model = torch.nn.Sequential(*model)
      else:
          model = call_obj(**model_cfg)
      model.apply(weights_init)

      # Magic
      wandb.watch(model)

      model = MMDataParallel(model, device_ids=range(gpus)).cuda()
      loss = call_obj(**loss_cfg)


      # build runner
      optimizer = call_obj(params=model.parameters(), **optimizer_cfg)
      runner = EpochBasedRunner(model, batch_processor, optimizer, work_dir, logger)
      runner.register_training_hooks(**training_hooks)

      if resume_from:
          runner.resume(resume_from)
      elif load_from:
          runner.load_checkpoint(load_from)

      # run
      workflow = [tuple(w) for w in workflow]
      runner.run(data_loaders, workflow, total_epochs, loss=loss)


# process a batch of data
def batch_processor(model, datas, train_mode):
    global loss

    data, label = datas
    data = data.cuda()
    label = label.cuda()

    # forward
    output = model(data)
    losses = loss(output, label)

    # output
    log_vars = dict(loss=losses.item())
    if not train_mode:
        log_vars['top1'] = topk_accuracy(output, label)
        log_vars['top3'] = topk_accuracy(output, label, 3)

    outputs = dict(loss=losses, log_vars=log_vars, num_samples=len(data.data))
    wandb.log(log_vars)
    return outputs


def topk_accuracy(score, label, k=1):
    rank = score.argsort()
    hit_top_k = [l in rank[i, -k:] for i, l in enumerate(label)]
    accuracy = sum(hit_top_k) * 1.0 / len(hit_top_k)
    return accuracy


def weights_init(model):
    classname = model.__class__.__name__
    if classname.find('Conv1d') != -1:
        model.weight.data.normal_(0.0, 0.02)
        if model.bias is not None:
            model.bias.data.fill_(0)
    elif classname.find('Conv2d') != -1:
        model.weight.data.normal_(0.0, 0.02)
        if model.bias is not None:
            model.bias.data.fill_(0)
    elif classname.find('BatchNorm') != -1:
        model.weight.data.normal_(1.0, 0.02)
        model.bias.data.fill_(0)


Overwriting mmskeleton/processor/recognition.py


### /content/mmskeleton/configs/pose_estimation/pose_demo.yaml

In [ ]:
%%writefile /content/mmskeleton/configs/pose_estimation/pose_demo.yaml
processor_cfg:
  type: "processor.pose_demo.inference"
  gpus: 1
  worker_per_gpu: 1
  video_file: resource/data_example/skateboarding.mp4
  save_dir: "work_dir/pose_demo"

  detection_cfg:
    model_cfg: /content/mmdetection/configs/cascade_rcnn/cascade_rcnn_r50_fpn_1x_coco.py
    checkpoint_file: http://download.openmmlab.com/mmdetection/v2.0/cascade_rcnn/cascade_rcnn_r50_fpn_1x_coco/cascade_rcnn_r50_fpn_1x_coco_20200316-3dc56deb.pth
    bbox_thre: 0.8
  estimation_cfg:
    model_cfg: configs/pose_estimation/hrnet/pose_hrnet_w32_256x192_test.yaml
    checkpoint_file: mmskeleton://pose_estimation/pose_hrnet_w32_256x192
    data_cfg:
      image_size:
        - 192
        - 256
      pixel_std: 200
      image_mean:
        - 0.485
        - 0.456
        - 0.406
      image_std:
        - 0.229
        - 0.224
        - 0.225
      post_process: true

argparse_cfg:
  gpus:
    bind_to: processor_cfg.gpus
    help: number of gpus
  video:
    bind_to: processor_cfg.video_file
    help: path to input video
  worker_per_gpu:
    bind_to: processor_cfg.worker_per_gpu
    help: number of workers for each gpu
  skeleton_model:
    bind_to: processor_cfg.estimation_cfg.model_cfg
  skeleton_checkpoint:
    bind_to: processor_cfg.estimation_cfg.checkpoint_file
  detection_model:
    bind_to: processor_cfg.detection_cfg.model_cfg
  detection_checkpoint:
    bind_to: processor_cfg.detection_cfg.checkpoint_file


Overwriting /content/mmskeleton/configs/pose_estimation/pose_demo.yaml


### /content/mmskeleton/configs/apis/pose_estimator.cascade_rcnn+hrnet.yaml

In [ ]:
%%writefile /content/mmskeleton/configs/apis/pose_estimator.cascade_rcnn+hrnet.yaml
detection_cfg:
  model_cfg: /content/mmdetection/configs/cascade_rcnn/cascade_rcnn_r50_fpn_1x_coco.py
  checkpoint_file: http://download.openmmlab.com/mmdetection/v2.0/cascade_rcnn/cascade_rcnn_r50_fpn_1x_coco/cascade_rcnn_r50_fpn_1x_coco_20200316-3dc56deb.pth
  bbox_thre: 0.8
estimation_cfg:
  model_cfg: configs/pose_estimation/hrnet/pose_hrnet_w32_256x192_test.yaml
  checkpoint_file: mmskeleton://pose_estimation/pose_hrnet_w32_256x192
  data_cfg:
    image_size:
      - 192
      - 256
    pixel_std: 200
    image_mean:
      - 0.485
      - 0.456
      - 0.406
    image_std:
      - 0.229
      - 0.224
      - 0.225
    post_process: true


Overwriting /content/mmskeleton/configs/apis/pose_estimator.cascade_rcnn+hrnet.yaml


### /content/mmskeleton/configs/utils/build_dataset_example.yaml

In [ ]:
%%writefile /content/mmskeleton/configs/utils/build_dataset_example.yaml
processor_cfg:
  type: "processor.skeleton_dataset.build"
  gpus: 1
  worker_per_gpu: 1
  video_dir: resource/data_example
  out_dir: "data/dataset_example"
  category_annotation: resource/category_annotation_example.json
  detection_cfg:
    model_cfg: /content/mmdetection/configs/cascade_rcnn/cascade_rcnn_r50_fpn_1x_coco.py
    checkpoint_file: http://download.openmmlab.com/mmdetection/v2.0/cascade_rcnn/cascade_rcnn_r50_fpn_1x_coco/cascade_rcnn_r50_fpn_1x_coco_20200316-3dc56deb.pth
    bbox_thre: 0.8
  estimation_cfg:
    model_cfg: configs/pose_estimation/hrnet/pose_hrnet_w32_256x192_test.yaml
    checkpoint_file: mmskeleton://pose_estimation/pose_hrnet_w32_256x192
    data_cfg:
      image_size:
        - 192
        - 256
      pixel_std: 200
      image_mean:
        - 0.485
        - 0.456
        - 0.406
      image_std:
        - 0.229
        - 0.224
        - 0.225
      post_process: true
  tracker_cfg: null

argparse_cfg:
  gpus:
    bind_to: processor_cfg.gpus
    help: number of gpus
  worker_per_gpu:
    bind_to: processor_cfg.worker_per_gpu
    help: number of workers for each gpu
  video_dir:
    bind_to: processor_cfg.video_dir
    help: folder for videos
  category_annotation:
    bind_to: processor_cfg.category_annotation
    help: a json file recording video category annotation
  out_dir:
    bind_to: processor_cfg.out_dir
    help: folder for storing output dataset
  skeleton_model:
    bind_to: processor_cfg.estimation_cfg.model_cfg
  skeleton_checkpoint:
    bind_to: processor_cfg.estimation_cfg.checkpoint_file
  detection_model:
    bind_to: processor_cfg.detection_cfg.model_cfg
  detection_checkpoint:
    bind_to: processor_cfg.detection_cfg.checkpoint_file

Overwriting /content/mmskeleton/configs/utils/build_dataset_example.yaml


### /content/mmskeleton/configs/utils/build_dataset_florence.yaml

In [ ]:
%%writefile /content/mmskeleton/configs/utils/build_dataset_florence.yaml
processor_cfg:
  type: "processor.skeleton_dataset.build"
  gpus: 1
  worker_per_gpu: 2
  video_dir: "/content/Florence_3d_actions"
  out_dir: "data/dataset_florence"
  category_annotation: resource/category_annotation_florence_ccam.json
  detection_cfg:
    model_cfg: /content/mmdetection/configs/cascade_rcnn/cascade_rcnn_r50_fpn_1x_coco.py
    checkpoint_file: http://download.openmmlab.com/mmdetection/v2.0/cascade_rcnn/cascade_rcnn_r50_fpn_1x_coco/cascade_rcnn_r50_fpn_1x_coco_20200316-3dc56deb.pth
    bbox_thre: 0.8
  estimation_cfg:
    model_cfg: configs/pose_estimation/hrnet/pose_hrnet_w32_256x192_test.yaml
    checkpoint_file: mmskeleton://pose_estimation/pose_hrnet_w32_256x192
    data_cfg:
      image_size:
        - 192
        - 256
      pixel_std: 200
      image_mean:
        - 0.485
        - 0.456
        - 0.406
      image_std:
        - 0.229
        - 0.224
        - 0.225
      post_process: true
  tracker_cfg: null

argparse_cfg:
  gpus:
    bind_to: processor_cfg.gpus
    help: number of gpus
  worker_per_gpu:
    bind_to: processor_cfg.worker_per_gpu
    help: number of workers for each gpu
  video_dir:
    bind_to: processor_cfg.video_dir
    help: folder for videos
  category_annotation:
    bind_to: processor_cfg.category_annotation
    help: a json file recording video category annotation
  out_dir:
    bind_to: processor_cfg.out_dir
    help: folder for storing output dataset
  skeleton_model:
    bind_to: processor_cfg.estimation_cfg.model_cfg
  skeleton_checkpoint:
    bind_to: processor_cfg.estimation_cfg.checkpoint_file
  detection_model:
    bind_to: processor_cfg.detection_cfg.model_cfg
  detection_checkpoint:
    bind_to: processor_cfg.detection_cfg.checkpoint_file

Writing /content/mmskeleton/configs/utils/build_dataset_florence.yaml


### /content/mmskeleton/configs/utils/build_dataset_ccam.yaml

In [ ]:
%%writefile /content/mmskeleton/configs/utils/build_dataset_ccam.yaml
processor_cfg:
  type: "processor.skeleton_dataset.build"
  gpus: 1
  worker_per_gpu: 2
  video_dir: "/content/ccam_actions"
  out_dir: "data/dataset_ccam"
  category_annotation: resource/category_annotation_florence_ccam.json
  detection_cfg:
    model_cfg: /content/mmdetection/configs/cascade_rcnn/cascade_rcnn_r50_fpn_1x_coco.py
    checkpoint_file: http://download.openmmlab.com/mmdetection/v2.0/cascade_rcnn/cascade_rcnn_r50_fpn_1x_coco/cascade_rcnn_r50_fpn_1x_coco_20200316-3dc56deb.pth
    bbox_thre: 0.8
  estimation_cfg:
    model_cfg: configs/pose_estimation/hrnet/pose_hrnet_w32_256x192_test.yaml
    checkpoint_file: mmskeleton://pose_estimation/pose_hrnet_w32_256x192
    data_cfg:
      image_size:
        - 192
        - 256
      pixel_std: 200
      image_mean:
        - 0.485
        - 0.456
        - 0.406
      image_std:
        - 0.229
        - 0.224
        - 0.225
      post_process: true
  tracker_cfg: null

argparse_cfg:
  gpus:
    bind_to: processor_cfg.gpus
    help: number of gpus
  worker_per_gpu:
    bind_to: processor_cfg.worker_per_gpu
    help: number of workers for each gpu
  video_dir:
    bind_to: processor_cfg.video_dir
    help: folder for videos
  category_annotation:
    bind_to: processor_cfg.category_annotation
    help: a json file recording video category annotation
  out_dir:
    bind_to: processor_cfg.out_dir
    help: folder for storing output dataset
  skeleton_model:
    bind_to: processor_cfg.estimation_cfg.model_cfg
  skeleton_checkpoint:
    bind_to: processor_cfg.estimation_cfg.checkpoint_file
  detection_model:
    bind_to: processor_cfg.detection_cfg.model_cfg
  detection_checkpoint:
    bind_to: processor_cfg.detection_cfg.checkpoint_file

Writing /content/mmskeleton/configs/utils/build_dataset_ccam.yaml


### /content/mmskeleton/resource/category_annotation_florence_ccam.json

In [ ]:
%%writefile /content/mmskeleton/resource/category_annotation_florence_ccam.json
{
    "categories": [
        "wave",
        "drink from a bottle",
        "answer phone",
        "clap",
        "tight lace",
        "sit down",
        "stand up",
        "read watch",
        "bow",
        "coming in",
        "going out",
        "g230"
    ],
    "annotations": {
        "GestureRecording_Id15actor7idAction15category7.avi": {
          "category_id": 6
        },
        "GestureRecording_Id7actor6idAction7category3.avi": {
          "category_id": 2
        },
        "GestureRecording_Id8actor9idAction8category4.avi": {
          "category_id": 3
        },
        "GestureRecording_Id19actor5idAction19category9.avi": {
          "category_id": 8
        },
        "GestureRecording_Id12actor9idAction12category5.avi": {
          "category_id": 4
        },
        "GestureRecording_Id7actor5idAction7category3.avi": {
          "category_id": 2
        },
        "GestureRecording_Id4actor5idAction4category2.avi": {
          "category_id": 1
        },
        "GestureRecording_Id5actor6idAction5category2.avi": {
          "category_id": 1
        },
        "GestureRecording_Id17actor9idAction17category8.avi": {
          "category_id": 7
        },
        "GestureRecording_Id15actor1idAction15category5.avi": {
          "category_id": 4
        },
        "GestureRecording_Id2actor5idAction2category1.avi": {
          "category_id": 0
        },
        "GestureRecording_Id4actor8idAction4category2.avi": {
          "category_id": 1
        },
        "GestureRecording_Id17actor1idAction17category5.avi": {
          "category_id": 4
        },
        "GestureRecording_Id2actor4idAction2category1.avi": {
          "category_id": 0
        },
        "GestureRecording_Id19actor4idAction19category9.avi": {
          "category_id": 8
        },
        "GestureRecording_Id22actor6idAction22category9.avi": {
          "category_id": 8
        },
        "GestureRecording_Id12actor7idAction12category6.avi": {
          "category_id": 5
        },
        "GestureRecording_Id25actor1idAction25category8.avi": {
          "category_id": 7
        },
        "GestureRecording_Id1actor1idAction1category1.avi": {
          "category_id": 0
        },
        "GestureRecording_Id10actor4idAction10category5.avi": {
          "category_id": 4
        },
        "GestureRecording_Id1actor2idAction1category1.avi": {
          "category_id": 0
        },
        "GestureRecording_Id13actor2idAction13category7.avi": {
          "category_id": 6
        },
        "GestureRecording_Id15actor5idAction15category7.avi": {
          "category_id": 6
        },
        "GestureRecording_Id7actor2idAction7category4.avi": {
          "category_id": 3
        },
        "GestureRecording_Id11actor7idAction11category5.avi": {
          "category_id": 4
        },
        "GestureRecording_Id28actor1idAction28category9.avi": {
          "category_id": 8
        },
        "GestureRecording_Id20actor9idAction20category9.avi": {
          "category_id": 8
        },
        "GestureRecording_Id9actor6idAction9category4.avi": {
          "category_id": 3
        },
        "GestureRecording_Id21actor6idAction21category9.avi": {
          "category_id": 8
        },
        "GestureRecording_Id5actor5idAction5category3.avi": {
          "category_id": 2
        },
        "GestureRecording_Id10actor9idAction10category4.avi": {
          "category_id": 3
        },
        "GestureRecording_Id14actor4idAction14category7.avi": {
          "category_id": 6
        },
        "GestureRecording_Id5actor2idAction5category3.avi": {
          "category_id": 2
        },
        "GestureRecording_Id9actor2idAction9category5.avi": {
          "category_id": 4
        },
        "GestureRecording_Id2actor10idAction2category1.avi": {
          "category_id": 0
        },
        "GestureRecording_Id6actor10idAction6category3.avi": {
          "category_id": 2
        },
        "GestureRecording_Id12actor10idAction12category5.avi": {
          "category_id": 4
        },
        "GestureRecording_Id2actor1idAction2category1.avi": {
          "category_id": 0
        },
        "GestureRecording_Id14actor3idAction14category7.avi": {
          "category_id": 6
        },
        "GestureRecording_Id17actor10idAction17category7.avi": {
          "category_id": 6
        },
        "GestureRecording_Id26actor1idAction26category9.avi": {
          "category_id": 8
        },
        "GestureRecording_Id8actor7idAction8category4.avi": {
          "category_id": 3
        },
        "GestureRecording_Id11actor5idAction11category5.avi": {
          "category_id": 4
        },
        "GestureRecording_Id3actor2idAction3category2.avi": {
          "category_id": 1
        },
        "GestureRecording_Id16actor2idAction16category8.avi": {
          "category_id": 7
        },
        "GestureRecording_Id4actor7idAction4category2.avi": {
          "category_id": 1
        },
        "GestureRecording_Id6actor3idAction6category3.avi": {
          "category_id": 2
        },
        "GestureRecording_Id2actor8idAction2category1.avi": {
          "category_id": 0
        },
        "GestureRecording_Id18actor6idAction18category8.avi": {
          "category_id": 7
        },
        "GestureRecording_Id14actor2idAction14category7.avi": {
          "category_id": 6
        },
        "GestureRecording_Id12actor5idAction12category6.avi": {
          "category_id": 5
        },
        "GestureRecording_Id14actor7idAction14category7.avi": {
          "category_id": 6
        },
        "GestureRecording_Id1actor4idAction1category1.avi": {
          "category_id": 0
        },
        "GestureRecording_Id5actor1idAction5category2.avi": {
          "category_id": 1
        },
        "GestureRecording_Id4actor3idAction4category2.avi": {
          "category_id": 1
        },
        "GestureRecording_Id12actor1idAction12category4.avi": {
          "category_id": 3
        },
        "GestureRecording_Id19actor9idAction19category9.avi": {
          "category_id": 8
        },
        "GestureRecording_Id16actor1idAction16category5.avi": {
          "category_id": 4
        },
        "GestureRecording_Id2actor3idAction2category1.avi": {
          "category_id": 0
        },
        "GestureRecording_Id16actor6idAction16category7.avi": {
          "category_id": 6
        },
        "GestureRecording_Id8actor1idAction8category3.avi": {
          "category_id": 2
        },
        "GestureRecording_Id14actor8idAction14category5.avi": {
          "category_id": 4
        },
        "GestureRecording_Id20actor5idAction20category9.avi": {
          "category_id": 8
        },
        "GestureRecording_Id2actor6idAction2category1.avi": {
          "category_id": 0
        },
        "GestureRecording_Id6actor2idAction6category3.avi": {
          "category_id": 2
        },
        "GestureRecording_Id1actor3idAction1category1.avi": {
          "category_id": 0
        },
        "GestureRecording_Id10actor3idAction10category5.avi": {
          "category_id": 4
        },
        "GestureRecording_Id3actor1idAction3category1.avi": {
          "category_id": 0
        },
        "GestureRecording_Id18actor8idAction18category7.avi": {
          "category_id": 6
        },
        "GestureRecording_Id3actor3idAction3category1.avi": {
          "category_id": 0
        },
        "GestureRecording_Id23actor8idAction23category9.avi": {
          "category_id": 8
        },
        "GestureRecording_Id4actor9idAction4category2.avi": {
          "category_id": 1
        },
        "GestureRecording_Id16actor4idAction16category8.avi": {
          "category_id": 7
        },
        "GestureRecording_Id15actor10idAction15category6.avi": {
          "category_id": 5
        },
        "GestureRecording_Id7actor1idAction7category3.avi": {
          "category_id": 2
        },
        "GestureRecording_Id6actor4idAction6category3.avi": {
          "category_id": 2
        },
        "GestureRecording_Id18actor7idAction18category9.avi": {
          "category_id": 8
        },
        "GestureRecording_Id9actor1idAction9category3.avi": {
          "category_id": 2
        },
        "GestureRecording_Id7actor8idAction7category4.avi": {
          "category_id": 3
        },
        "GestureRecording_Id14actor1idAction14category5.avi": {
          "category_id": 4
        },
        "GestureRecording_Id23actor1idAction23category8.avi": {
          "category_id": 7
        },
        "GestureRecording_Id12actor3idAction12category6.avi": {
          "category_id": 5
        },
        "GestureRecording_Id13actor9idAction13category6.avi": {
          "category_id": 5
        },
        "GestureRecording_Id18actor4idAction18category9.avi": {
          "category_id": 8
        },
        "GestureRecording_Id22actor1idAction22category8.avi": {
          "category_id": 7
        },
        "GestureRecording_Id9actor10idAction9category4.avi": {
          "category_id": 3
        },
        "GestureRecording_Id3actor5idAction3category2.avi": {
          "category_id": 1
        },
        "GestureRecording_Id17actor5idAction17category8.avi": {
          "category_id": 7
        },
        "GestureRecording_Id11actor6idAction11category5.avi": {
          "category_id": 4
        },
        "GestureRecording_Id7actor7idAction7category4.avi": {
          "category_id": 3
        },
        "GestureRecording_Id13actor7idAction13category6.avi": {
          "category_id": 5
        },
        "GestureRecording_Id13actor10idAction13category5.avi": {
          "category_id": 4
        },
        "GestureRecording_Id17actor3idAction17category8.avi": {
          "category_id": 7
        },
        "GestureRecording_Id10actor1idAction10category4.avi": {
          "category_id": 3
        },
        "GestureRecording_Id4actor10idAction4category2.avi": {
          "category_id": 1
        },
        "GestureRecording_Id21actor10idAction21category9.avi": {
          "category_id": 8
        },
        "GestureRecording_Id5actor10idAction5category3.avi": {
          "category_id": 2
        },
        "GestureRecording_Id6actor7idAction6category3.avi": {
          "category_id": 2
        },
        "GestureRecording_Id15actor3idAction15category7.avi": {
          "category_id": 6
        },
        "GestureRecording_Id1actor8idAction1category1.avi": {
          "category_id": 0
        },
        "GestureRecording_Id22actor10idAction22category9.avi": {
          "category_id": 8
        },
        "GestureRecording_Id10actor5idAction10category5.avi": {
          "category_id": 4
        },
        "GestureRecording_Id16actor10idAction16category7.avi": {
          "category_id": 6
        },
        "GestureRecording_Id7actor9idAction7category4.avi": {
          "category_id": 3
        },
        "GestureRecording_Id3actor10idAction3category2.avi": {
          "category_id": 1
        },
        "GestureRecording_Id20actor3idAction20category9.avi": {
          "category_id": 8
        },
        "GestureRecording_Id19actor3idAction19category9.avi": {
          "category_id": 8
        },
        "GestureRecording_Id4actor6idAction4category2.avi": {
          "category_id": 1
        },
        "GestureRecording_Id27actor1idAction27category9.avi": {
          "category_id": 8
        },
        "GestureRecording_Id20actor10idAction20category9.avi": {
          "category_id": 8
        },
        "GestureRecording_Id6actor8idAction6category3.avi": {
          "category_id": 2
        },
        "GestureRecording_Id13actor8idAction13category5.avi": {
          "category_id": 4
        },
        "GestureRecording_Id8actor3idAction8category4.avi": {
          "category_id": 3
        },
        "GestureRecording_Id19actor1idAction19category6.avi": {
          "category_id": 5
        },
        "GestureRecording_Id1actor9idAction1category1.avi": {
          "category_id": 0
        },
        "GestureRecording_Id18actor10idAction18category8.avi": {
          "category_id": 7
        },
        "GestureRecording_Id11actor8idAction11category5.avi": {
          "category_id": 4
        },
        "GestureRecording_Id11actor4idAction11category5.avi": {
          "category_id": 4
        },
        "GestureRecording_Id13actor3idAction13category6.avi": {
          "category_id": 5
        },
        "GestureRecording_Id2actor2idAction2category1.avi": {
          "category_id": 0
        },
        "GestureRecording_Id12actor8idAction12category5.avi": {
          "category_id": 4
        },
        "GestureRecording_Id9actor3idAction9category4.avi": {
          "category_id": 3
        },
        "GestureRecording_Id1actor6idAction1category1.avi": {
          "category_id": 0
        },
        "GestureRecording_Id16actor8idAction16category6.avi": {
          "category_id": 5
        },
        "GestureRecording_Id8actor5idAction8category4.avi": {
          "category_id": 3
        },
        "GestureRecording_Id19actor7idAction19category9.avi": {
          "category_id": 8
        },
        "GestureRecording_Id16actor5idAction16category8.avi": {
          "category_id": 7
        },
        "GestureRecording_Id17actor4idAction17category8.avi": {
          "category_id": 7
        },
        "GestureRecording_Id16actor3idAction16category8.avi": {
          "category_id": 7
        },
        "GestureRecording_Id3actor6idAction3category1.avi": {
          "category_id": 0
        },
        "GestureRecording_Id19actor10idAction19category8.avi": {
          "category_id": 7
        },
        "GestureRecording_Id9actor5idAction9category4.avi": {
          "category_id": 3
        },
        "GestureRecording_Id15actor6idAction15category7.avi": {
          "category_id": 6
        },
        "GestureRecording_Id3actor8idAction3category2.avi": {
          "category_id": 1
        },
        "GestureRecording_Id18actor1idAction18category6.avi": {
          "category_id": 5
        },
        "GestureRecording_Id6actor6idAction6category3.avi": {
          "category_id": 2
        },
        "GestureRecording_Id15actor4idAction15category7.avi": {
          "category_id": 6
        },
        "GestureRecording_Id7actor3idAction7category3.avi": {
          "category_id": 2
        },
        "GestureRecording_Id13actor6idAction13category6.avi": {
          "category_id": 5
        },
        "GestureRecording_Id19actor6idAction19category9.avi": {
          "category_id": 8
        },
        "GestureRecording_Id6actor5idAction6category3.avi": {
          "category_id": 2
        },
        "GestureRecording_Id14actor6idAction14category6.avi": {
          "category_id": 5
        },
        "GestureRecording_Id10actor8idAction10category4.avi": {
          "category_id": 3
        },
        "GestureRecording_Id19actor8idAction19category8.avi": {
          "category_id": 7
        },
        "GestureRecording_Id10actor7idAction10category5.avi": {
          "category_id": 4
        },
        "GestureRecording_Id13actor4idAction13category6.avi": {
          "category_id": 5
        },
        "GestureRecording_Id4actor1idAction4category2.avi": {
          "category_id": 1
        },
        "GestureRecording_Id8actor10idAction8category4.avi": {
          "category_id": 3
        },
        "GestureRecording_Id17actor2idAction17category9.avi": {
          "category_id": 8
        },
        "GestureRecording_Id20actor1idAction20category7.avi": {
          "category_id": 6
        },
        "GestureRecording_Id4actor4idAction4category2.avi": {
          "category_id": 1
        },
        "GestureRecording_Id22actor8idAction22category9.avi": {
          "category_id": 8
        },
        "GestureRecording_Id12actor2idAction12category6.avi": {
          "category_id": 5
        },
        "GestureRecording_Id11actor10idAction11category4.avi": {
          "category_id": 3
        },
        "GestureRecording_Id7actor10idAction7category4.avi": {
          "category_id": 3
        },
        "GestureRecording_Id5actor4idAction5category2.avi": {
          "category_id": 1
        },
        "GestureRecording_Id4actor2idAction4category2.avi": {
          "category_id": 1
        },
        "GestureRecording_Id12actor6idAction12category5.avi": {
          "category_id": 4
        },
        "GestureRecording_Id24actor1idAction24category8.avi": {
          "category_id": 7
        },
        "GestureRecording_Id14actor10idAction14category6.avi": {
          "category_id": 5
        },
        "GestureRecording_Id23actor10idAction23category9.avi": {
          "category_id": 8
        },
        "GestureRecording_Id18actor5idAction18category9.avi": {
          "category_id": 8
        },
        "GestureRecording_Id17actor7idAction17category8.avi": {
          "category_id": 7
        },
        "GestureRecording_Id6actor1idAction6category2.avi": {
          "category_id": 1
        },
        "GestureRecording_Id9actor7idAction9category5.avi": {
          "category_id": 4
        },
        "GestureRecording_Id3actor4idAction3category1.avi": {
          "category_id": 0
        },
        "GestureRecording_Id16actor9idAction16category7.avi": {
          "category_id": 6
        },
        "GestureRecording_Id17actor6idAction17category8.avi": {
          "category_id": 7
        },
        "GestureRecording_Id5actor8idAction5category3.avi": {
          "category_id": 2
        },
        "GestureRecording_Id2actor9idAction2category1.avi": {
          "category_id": 0
        },
        "GestureRecording_Id12actor4idAction12category6.avi": {
          "category_id": 5
        },
        "GestureRecording_Id11actor1idAction11category4.avi": {
          "category_id": 3
        },
        "GestureRecording_Id7actor4idAction7category3.avi": {
          "category_id": 2
        },
        "GestureRecording_Id14actor9idAction14category6.avi": {
          "category_id": 5
        },
        "GestureRecording_Id8actor2idAction8category4.avi": {
          "category_id": 3
        },
        "GestureRecording_Id21actor8idAction21category9.avi": {
          "category_id": 8
        },
        "GestureRecording_Id11actor3idAction11category5.avi": {
          "category_id": 4
        },
        "GestureRecording_Id3actor9idAction3category2.avi": {
          "category_id": 1
        },
        "GestureRecording_Id8actor8idAction8category4.avi": {
          "category_id": 3
        },
        "GestureRecording_Id18actor9idAction18category8.avi": {
          "category_id": 7
        },
        "GestureRecording_Id8actor4idAction8category4.avi": {
          "category_id": 3
        },
        "GestureRecording_Id5actor9idAction5category3.avi": {
          "category_id": 2
        },
        "GestureRecording_Id29actor1idAction29category9.avi": {
          "category_id": 8
        },
        "GestureRecording_Id9actor9idAction9category4.avi": {
          "category_id": 3
        },
        "GestureRecording_Id13actor5idAction13category6.avi": {
          "category_id": 5
        },
        "GestureRecording_Id10actor2idAction10category5.avi": {
          "category_id": 4
        },
        "GestureRecording_Id16actor7idAction16category8.avi": {
          "category_id": 7
        },
        "GestureRecording_Id10actor10idAction10category4.avi": {
          "category_id": 3
        },
        "GestureRecording_Id15actor9idAction15category7.avi": {
          "category_id": 6
        },
        "GestureRecording_Id13actor1idAction13category4.avi": {
          "category_id": 3
        },
        "GestureRecording_Id1actor10idAction1category1.avi": {
          "category_id": 0
        },
        "GestureRecording_Id15actor2idAction15category8.avi": {
          "category_id": 7
        },
        "GestureRecording_Id11actor2idAction11category6.avi": {
          "category_id": 5
        },
        "GestureRecording_Id15actor8idAction15category6.avi": {
          "category_id": 5
        },
        "GestureRecording_Id6actor9idAction6category3.avi": {
          "category_id": 2
        },
        "GestureRecording_Id9actor4idAction9category4.avi": {
          "category_id": 3
        },
        "GestureRecording_Id2actor7idAction2category1.avi": {
          "category_id": 0
        },
        "GestureRecording_Id20actor6idAction20category9.avi": {
          "category_id": 8
        },
        "GestureRecording_Id3actor7idAction3category2.avi": {
          "category_id": 1
        },
        "GestureRecording_Id18actor3idAction18category8.avi": {
          "category_id": 7
        },
        "GestureRecording_Id1actor7idAction1category1.avi": {
          "category_id": 0
        },
        "GestureRecording_Id20actor8idAction20category8.avi": {
          "category_id": 7
        },
        "GestureRecording_Id11actor9idAction11category5.avi": {
          "category_id": 4
        },
        "GestureRecording_Id9actor8idAction9category4.avi": {
          "category_id": 3
        },
        "GestureRecording_Id8actor6idAction8category4.avi": {
          "category_id": 3
        },
        "GestureRecording_Id18actor2idAction18category9.avi": {
          "category_id": 8
        },
        "GestureRecording_Id5actor3idAction5category2.avi": {
          "category_id": 1
        },
        "GestureRecording_Id1actor5idAction1category1.avi": {
          "category_id": 0
        },
        "GestureRecording_Id10actor6idAction10category5.avi": {
          "category_id": 4
        },
        "GestureRecording_Id14actor5idAction14category7.avi": {
          "category_id": 6
        },
        "GestureRecording_Id5actor7idAction5category3.avi": {
          "category_id": 2
        },
        "GestureRecording_Id21actor3idAction21category9.avi": {
          "category_id": 8
        },
        "GestureRecording_Id17actor8idAction17category7.avi": {
          "category_id": 6
        },
        "GestureRecording_Id24actor8idAction24category9.avi": {
          "category_id": 8
        },
        "GestureRecording_Id21actor1idAction21category7.avi": {
          "category_id": 6
        },
        "TightLace_cam_rosberry1_color_image_raw_act2.mp4": {
          "category_id": 4
        },
        "Clapping_cam_rosberry3_color_image_raw_comingin.mp4": {
          "category_id": 9
        },
        "SitDownStandUp_cam_rosberry2_color_image_raw_act1.mp4": {
          "category_id": 5
        },
        "ReadWatch_cam_rosberry4_color_image_raw_goingout.mp4": {
          "category_id": 10
        },
        "Clapping_cam_rosberry4_color_image_raw_goingout.mp4": {
          "category_id": 10
        },
        "DrinkFromBottle_cam_rosberry4_color_image_raw_act1.mp4": {
          "category_id": 1
        },
        "AnswerPhone_cam_rosberry4_color_image_raw_act3.mp4": {
          "category_id": 2
        },
        "Bow_cam_rosberry2_color_image_raw_goingout.mp4": {
          "category_id": 10
        },
        "WaveHand_cam_rosberry2_color_image_raw_goingout.mp4": {
          "category_id": 10
        },
        "G230_Master_cam_rosberry2_color_image_raw_act3.mp4": {
          "category_id": 11
        },
        "SitDownStandUp_cam_rosberry4_color_image_raw_goingout.mp4": {
          "category_id": 10
        },
        "Clapping_cam_rosberry1_color_image_raw_act1.mp4": {
          "category_id": 3
        },
        "Stationary_cam_rosberry3_color_image_raw_comingin.mp4": {
          "category_id": 9
        },
        "G230_Master_cam_rosberry1_color_image_raw_act1.mp4": {
          "category_id": 11
        },
        "DrinkFromBottle_cam_rosberry1_color_image_raw_act2.mp4": {
          "category_id": 1
        },
        "TightLace_cam_rosberry2_color_image_raw_act2.mp4": {
          "category_id": 4
        },
        "AnswerPhone_cam_rosberry4_color_image_raw_comingin.mp4": {
          "category_id": 9
        },
        "G230_Master_cam_rosberry3_color_image_raw_comingin.mp4": {
          "category_id": 9
        },
        "SitDownStandUp_cam_rosberry4_color_image_raw_act2.mp4": {
          "category_id": 5
        },
        "TightLace_cam_rosberry3_color_image_raw_act3.mp4": {
          "category_id": 4
        },
        "G230_Master_cam_rosberry2_color_image_raw_comingin.mp4": {
          "category_id": 9
        },
        "Stationary_cam_rosberry2_color_image_raw_act2.mp4": {
          "category_id": 6
        },
        "Clapping_cam_rosberry2_color_image_raw_act2.mp4": {
          "category_id": 3
        },
        "ReadWatch_cam_rosberry3_color_image_raw_act2.mp4": {
          "category_id": 7
        },
        "WaveHand_cam_rosberry4_color_image_raw_goingout.mp4": {
          "category_id": 10
        },
        "Stationary_cam_rosberry3_color_image_raw_act1.mp4": {
          "category_id": 6
        },
        "TightLace_cam_rosberry3_color_image_raw_act1.mp4": {
          "category_id": 4
        },
        "ReadWatch_cam_rosberry1_color_image_raw_goingout.mp4": {
          "category_id": 10
        },
        "DrinkFromBottle_cam_rosberry2_color_image_raw_act3.mp4": {
          "category_id": 1
        },
        "ReadWatch_cam_rosberry4_color_image_raw_act2.mp4": {
          "category_id": 7
        },
        "Clapping_cam_rosberry1_color_image_raw_comingin.mp4": {
          "category_id": 9
        },
        "TightLace_cam_rosberry1_color_image_raw_act3.mp4": {
          "category_id": 4
        },
        "Bow_cam_rosberry4_color_image_raw_comingin.mp4": {
          "category_id": 9
        },
        "SitDownStandUp_cam_rosberry1_color_image_raw_act2.mp4": {
          "category_id": 5
        },
        "ReadWatch_cam_rosberry2_color_image_raw_act1.mp4": {
          "category_id": 7
        },
        "Bow_cam_rosberry2_color_image_raw_comingin.mp4": {
          "category_id": 9
        },
        "TightLace_cam_rosberry4_color_image_raw_act2.mp4": {
          "category_id": 4
        },
        "Bow_cam_rosberry4_color_image_raw_act1.mp4": {
          "category_id": 8
        },
        "SitDownStandUp_cam_rosberry2_color_image_raw_goingout.mp4": {
          "category_id": 10
        },
        "DrinkFromBottle_cam_rosberry3_color_image_raw_act3.mp4": {
          "category_id": 1
        },
        "TightLace_cam_rosberry4_color_image_raw_act3.mp4": {
          "category_id": 4
        },
        "Stationary_cam_rosberry3_color_image_raw_act3.mp4": {
          "category_id": 6
        },
        "DrinkFromBottle_cam_rosberry4_color_image_raw_act2.mp4": {
          "category_id": 1
        },
        "Stationary_cam_rosberry3_color_image_raw_act2.mp4": {
          "category_id": 6
        },
        "AnswerPhone_cam_rosberry3_color_image_raw_act3.mp4": {
          "category_id": 2
        },
        "AnswerPhone_cam_rosberry3_color_image_raw_comingin.mp4": {
          "category_id": 9
        },
        "G230_Master_cam_rosberry4_color_image_raw_goingout.mp4": {
          "category_id": 10
        },
        "Stationary_cam_rosberry1_color_image_raw_goingout.mp4": {
          "category_id": 10
        },
        "WaveHand_cam_rosberry1_color_image_raw_act2.mp4": {
          "category_id": 0
        },
        "SitDownStandUp_cam_rosberry3_color_image_raw_goingout.mp4": {
          "category_id": 10
        },
        "DrinkFromBottle_cam_rosberry4_color_image_raw_comingin.mp4": {
          "category_id": 9
        },
        "Bow_cam_rosberry2_color_image_raw_act2.mp4": {
          "category_id": 8
        },
        "Stationary_cam_rosberry2_color_image_raw_comingin.mp4": {
          "category_id": 9
        },
        "ReadWatch_cam_rosberry3_color_image_raw_goingout.mp4": {
          "category_id": 10
        },
        "Clapping_cam_rosberry2_color_image_raw_act1.mp4": {
          "category_id": 3
        },
        "Bow_cam_rosberry4_color_image_raw_goingout.mp4": {
          "category_id": 10
        },
        "WaveHand_cam_rosberry2_color_image_raw_act1.mp4": {
          "category_id": 0
        },
        "SitDownStandUp_cam_rosberry3_color_image_raw_act2.mp4": {
          "category_id": 5
        },
        "DrinkFromBottle_cam_rosberry3_color_image_raw_goingout.mp4": {
          "category_id": 10
        },
        "AnswerPhone_cam_rosberry1_color_image_raw_act1.mp4": {
          "category_id": 2
        },
        "G230_Master_cam_rosberry4_color_image_raw_act1.mp4": {
          "category_id": 11
        },
        "SitDownStandUp_cam_rosberry1_color_image_raw_act1.mp4": {
          "category_id": 5
        },
        "SitDownStandUp_cam_rosberry4_color_image_raw_comingin.mp4": {
          "category_id": 9
        },
        "G230_Master_cam_rosberry1_color_image_raw_act2.mp4": {
          "category_id": 11
        },
        "G230_Master_cam_rosberry3_color_image_raw_act3.mp4": {
          "category_id": 11
        },
        "TightLace_cam_rosberry4_color_image_raw_act1.mp4": {
          "category_id": 4
        },
        "Stationary_cam_rosberry4_color_image_raw_act3.mp4": {
          "category_id": 6
        },
        "SitDownStandUp_cam_rosberry2_color_image_raw_comingin.mp4": {
          "category_id": 9
        },
        "AnswerPhone_cam_rosberry2_color_image_raw_goingout.mp4": {
          "category_id": 10
        },
        "Clapping_cam_rosberry4_color_image_raw_comingin.mp4": {
          "category_id": 9
        },
        "Stationary_cam_rosberry1_color_image_raw_comingin.mp4": {
          "category_id": 9
        },
        "Bow_cam_rosberry1_color_image_raw_goingout.mp4": {
          "category_id": 10
        },
        "WaveHand_cam_rosberry3_color_image_raw_act1.mp4": {
          "category_id": 0
        },
        "Bow_cam_rosberry1_color_image_raw_act2.mp4": {
          "category_id": 8
        },
        "AnswerPhone_cam_rosberry2_color_image_raw_act3.mp4": {
          "category_id": 2
        },
        "G230_Master_cam_rosberry1_color_image_raw_comingin.mp4": {
          "category_id": 9
        },
        "DrinkFromBottle_cam_rosberry3_color_image_raw_act2.mp4": {
          "category_id": 1
        },
        "AnswerPhone_cam_rosberry3_color_image_raw_act2.mp4": {
          "category_id": 2
        },
        "WaveHand_cam_rosberry2_color_image_raw_act2.mp4": {
          "category_id": 0
        },
        "AnswerPhone_cam_rosberry2_color_image_raw_act1.mp4": {
          "category_id": 2
        },
        "ReadWatch_cam_rosberry3_color_image_raw_comingin.mp4": {
          "category_id": 9
        },
        "WaveHand_cam_rosberry3_color_image_raw_goingout.mp4": {
          "category_id": 10
        },
        "ReadWatch_cam_rosberry1_color_image_raw_act1.mp4": {
          "category_id": 7
        },
        "Clapping_cam_rosberry3_color_image_raw_act1.mp4": {
          "category_id": 3
        },
        "Stationary_cam_rosberry4_color_image_raw_act2.mp4": {
          "category_id": 6
        },
        "Bow_cam_rosberry2_color_image_raw_act1.mp4": {
          "category_id": 8
        },
        "AnswerPhone_cam_rosberry2_color_image_raw_comingin.mp4": {
          "category_id": 9
        },
        "DrinkFromBottle_cam_rosberry3_color_image_raw_act1.mp4": {
          "category_id": 1
        },
        "TightLace_cam_rosberry2_color_image_raw_goingout.mp4": {
          "category_id": 10
        },
        "WaveHand_cam_rosberry3_color_image_raw_comingin.mp4": {
          "category_id": 9
        },
        "Stationary_cam_rosberry1_color_image_raw_act2.mp4": {
          "category_id": 6
        },
        "TightLace_cam_rosberry4_color_image_raw_goingout.mp4": {
          "category_id": 10
        },
        "G230_Master_cam_rosberry2_color_image_raw_goingout.mp4": {
          "category_id": 10
        },
        "Clapping_cam_rosberry1_color_image_raw_act2.mp4": {
          "category_id": 3
        },
        "SitDownStandUp_cam_rosberry3_color_image_raw_act1.mp4": {
          "category_id": 5
        },
        "AnswerPhone_cam_rosberry2_color_image_raw_act2.mp4": {
          "category_id": 2
        },
        "Bow_cam_rosberry1_color_image_raw_act1.mp4": {
          "category_id": 8
        },
        "Stationary_cam_rosberry4_color_image_raw_act1.mp4": {
          "category_id": 6
        },
        "Stationary_cam_rosberry2_color_image_raw_goingout.mp4": {
          "category_id": 10
        },
        "AnswerPhone_cam_rosberry3_color_image_raw_act1.mp4": {
          "category_id": 2
        },
        "WaveHand_cam_rosberry2_color_image_raw_comingin.mp4": {
          "category_id": 9
        },
        "ReadWatch_cam_rosberry1_color_image_raw_act2.mp4": {
          "category_id": 7
        },
        "WaveHand_cam_rosberry4_color_image_raw_comingin.mp4": {
          "category_id": 9
        },
        "TightLace_cam_rosberry1_color_image_raw_goingout.mp4": {
          "category_id": 10
        },
        "Bow_cam_rosberry1_color_image_raw_comingin.mp4": {
          "category_id": 9
        },
        "ReadWatch_cam_rosberry2_color_image_raw_act2.mp4": {
          "category_id": 7
        },
        "ReadWatch_cam_rosberry4_color_image_raw_act1.mp4": {
          "category_id": 7
        },
        "AnswerPhone_cam_rosberry1_color_image_raw_act2.mp4": {
          "category_id": 2
        },
        "TightLace_cam_rosberry3_color_image_raw_comingin.mp4": {
          "category_id": 9
        },
        "Bow_cam_rosberry3_color_image_raw_comingin.mp4": {
          "category_id": 9
        },
        "DrinkFromBottle_cam_rosberry2_color_image_raw_act1.mp4": {
          "category_id": 1
        },
        "DrinkFromBottle_cam_rosberry2_color_image_raw_act2.mp4": {
          "category_id": 1
        },
        "Clapping_cam_rosberry2_color_image_raw_comingin.mp4": {
          "category_id": 9
        },
        "Stationary_cam_rosberry2_color_image_raw_act1.mp4": {
          "category_id": 6
        },
        "G230_Master_cam_rosberry3_color_image_raw_act2.mp4": {
          "category_id": 11
        },
        "Bow_cam_rosberry3_color_image_raw_goingout.mp4": {
          "category_id": 10
        },
        "DrinkFromBottle_cam_rosberry2_color_image_raw_goingout.mp4": {
          "category_id": 10
        },
        "AnswerPhone_cam_rosberry1_color_image_raw_comingin.mp4": {
          "category_id": 9
        },
        "TightLace_cam_rosberry1_color_image_raw_act1.mp4": {
          "category_id": 4
        },
        "DrinkFromBottle_cam_rosberry2_color_image_raw_comingin.mp4": {
          "category_id": 9
        },
        "Stationary_cam_rosberry1_color_image_raw_act3.mp4": {
          "category_id": 6
        },
        "WaveHand_cam_rosberry1_color_image_raw_comingin.mp4": {
          "category_id": 9
        },
        "G230_Master_cam_rosberry4_color_image_raw_act2.mp4": {
          "category_id": 11
        },
        "G230_Master_cam_rosberry2_color_image_raw_act1.mp4": {
          "category_id": 11
        },
        "SitDownStandUp_cam_rosberry1_color_image_raw_goingout.mp4": {
          "category_id": 10
        },
        "Clapping_cam_rosberry4_color_image_raw_act1.mp4": {
          "category_id": 3
        },
        "TightLace_cam_rosberry3_color_image_raw_act2.mp4": {
          "category_id": 4
        },
        "G230_Master_cam_rosberry1_color_image_raw_goingout.mp4": {
          "category_id": 10
        },
        "G230_Master_cam_rosberry3_color_image_raw_act1.mp4": {
          "category_id": 11
        },
        "AnswerPhone_cam_rosberry1_color_image_raw_goingout.mp4": {
          "category_id": 10
        },
        "ReadWatch_cam_rosberry3_color_image_raw_act1.mp4": {
          "category_id": 7
        },
        "Stationary_cam_rosberry4_color_image_raw_goingout.mp4": {
          "category_id": 10
        },
        "DrinkFromBottle_cam_rosberry1_color_image_raw_act1.mp4": {
          "category_id": 1
        },
        "DrinkFromBottle_cam_rosberry1_color_image_raw_goingout.mp4": {
          "category_id": 10
        },
        "G230_Master_cam_rosberry4_color_image_raw_comingin.mp4": {
          "category_id": 9
        },
        "Bow_cam_rosberry3_color_image_raw_act1.mp4": {
          "category_id": 8
        },
        "G230_Master_cam_rosberry2_color_image_raw_act2.mp4": {
          "category_id": 11
        },
        "Stationary_cam_rosberry2_color_image_raw_act3.mp4": {
          "category_id": 6
        },
        "Stationary_cam_rosberry4_color_image_raw_comingin.mp4": {
          "category_id": 9
        },
        "ReadWatch_cam_rosberry4_color_image_raw_comingin.mp4": {
          "category_id": 9
        },
        "AnswerPhone_cam_rosberry1_color_image_raw_act3.mp4": {
          "category_id": 2
        },
        "WaveHand_cam_rosberry3_color_image_raw_act3.mp4": {
          "category_id": 0
        },
        "ReadWatch_cam_rosberry1_color_image_raw_comingin.mp4": {
          "category_id": 9
        },
        "TightLace_cam_rosberry2_color_image_raw_act1.mp4": {
          "category_id": 4
        },
        "WaveHand_cam_rosberry4_color_image_raw_act2.mp4": {
          "category_id": 0
        },
        "DrinkFromBottle_cam_rosberry1_color_image_raw_comingin.mp4": {
          "category_id": 9
        },
        "ReadWatch_cam_rosberry2_color_image_raw_comingin.mp4": {
          "category_id": 9
        },
        "AnswerPhone_cam_rosberry4_color_image_raw_act1.mp4": {
          "category_id": 2
        },
        "AnswerPhone_cam_rosberry4_color_image_raw_act2.mp4": {
          "category_id": 2
        },
        "Bow_cam_rosberry4_color_image_raw_act2.mp4": {
          "category_id": 8
        },
        "SitDownStandUp_cam_rosberry4_color_image_raw_act1.mp4": {
          "category_id": 5
        },
        "Clapping_cam_rosberry3_color_image_raw_goingout.mp4": {
          "category_id": 10
        },
        "AnswerPhone_cam_rosberry4_color_image_raw_goingout.mp4": {
          "category_id": 10
        },
        "ReadWatch_cam_rosberry2_color_image_raw_goingout.mp4": {
          "category_id": 10
        },
        "Clapping_cam_rosberry2_color_image_raw_goingout.mp4": {
          "category_id": 10
        },
        "TightLace_cam_rosberry2_color_image_raw_comingin.mp4": {
          "category_id": 9
        },
        "DrinkFromBottle_cam_rosberry4_color_image_raw_act3.mp4": {
          "category_id": 1
        },
        "Clapping_cam_rosberry3_color_image_raw_act2.mp4": {
          "category_id": 3
        },
        "WaveHand_cam_rosberry2_color_image_raw_act3.mp4": {
          "category_id": 0
        },
        "Bow_cam_rosberry3_color_image_raw_act2.mp4": {
          "category_id": 8
        },
        "Clapping_cam_rosberry4_color_image_raw_act2.mp4": {
          "category_id": 3
        },
        "SitDownStandUp_cam_rosberry3_color_image_raw_comingin.mp4": {
          "category_id": 9
        },
        "TightLace_cam_rosberry2_color_image_raw_act3.mp4": {
          "category_id": 4
        },
        "G230_Master_cam_rosberry1_color_image_raw_act3.mp4": {
          "category_id": 11
        },
        "AnswerPhone_cam_rosberry3_color_image_raw_goingout.mp4": {
          "category_id": 10
        },
        "Clapping_cam_rosberry1_color_image_raw_goingout.mp4": {
          "category_id": 10
        },
        "WaveHand_cam_rosberry3_color_image_raw_act2.mp4": {
          "category_id": 0
        },
        "WaveHand_cam_rosberry4_color_image_raw_act3.mp4": {
          "category_id": 0
        },
        "G230_Master_cam_rosberry3_color_image_raw_goingout.mp4": {
          "category_id": 10
        },
        "TightLace_cam_rosberry1_color_image_raw_comingin.mp4": {
          "category_id": 9
        },
        "WaveHand_cam_rosberry4_color_image_raw_act1.mp4": {
          "category_id": 0
        },
        "TightLace_cam_rosberry3_color_image_raw_goingout.mp4": {
          "category_id": 10
        },
        "WaveHand_cam_rosberry1_color_image_raw_act3.mp4": {
          "category_id": 0
        },
        "WaveHand_cam_rosberry1_color_image_raw_goingout.mp4": {
          "category_id": 10
        },
        "SitDownStandUp_cam_rosberry2_color_image_raw_act2.mp4": {
          "category_id": 5
        },
        "DrinkFromBottle_cam_rosberry3_color_image_raw_comingin.mp4": {
          "category_id": 9
        },
        "G230_Master_cam_rosberry4_color_image_raw_act3.mp4": {
          "category_id": 11
        },
        "DrinkFromBottle_cam_rosberry1_color_image_raw_act3.mp4": {
          "category_id": 1
        },
        "DrinkFromBottle_cam_rosberry4_color_image_raw_goingout.mp4": {
          "category_id": 10
        },
        "WaveHand_cam_rosberry1_color_image_raw_act1.mp4": {
          "category_id": 0
        },
        "Stationary_cam_rosberry1_color_image_raw_act1.mp4": {
          "category_id": 6
        },
        "Stationary_cam_rosberry3_color_image_raw_goingout.mp4": {
          "category_id": 10
        },
        "TightLace_cam_rosberry4_color_image_raw_comingin.mp4": {
          "category_id": 9
        },
        "SitDownStandUp_cam_rosberry1_color_image_raw_comingin.mp4": {
          "category_id": 9
        }
    }
}


Writing /content/mmskeleton/resource/category_annotation_florence_ccam.json


### pose demo

In [ ]:
# %cd /content/mmskeleton
# !mmskl pose_demo

/content/mmskeleton
INFO:root:Generating grammar tables from /usr/lib/python3.6/lib2to3/Grammar.txt
INFO:root:Generating grammar tables from /usr/lib/python3.6/lib2to3/PatternGrammar.txt
/usr/local/lib/python3.6/dist-packages/Cython/Distutils/old_build_ext.py:41: UserWarning: Cython.Distutils.old_build_ext does not properly handle dependencies and is deprecated.
  "Cython.Distutils.old_build_ext does not properly handle dependencies "
Load configuration information from ./configs/pose_estimation/pose_demo.yaml

Pose estimation:
Downloading: "http://download.openmmlab.com/mmdetection/v2.0/cascade_rcnn/cascade_rcnn_r50_fpn_1x_coco/cascade_rcnn_r50_fpn_1x_coco_20200316-3dc56deb.pth" to /root/.cache/torch/hub/checkpoints/cascade_rcnn_r50_fpn_1x_coco_20200316-3dc56deb.pth
100% 265M/265M [00:22<00:00, 12.2MB/s]
Downloading: "https://open-mmlab.s3.ap-northeast-2.amazonaws.com/mmskeleton/models/pose_estimation/pose_hrnet_w32_256x192-76ea353b.pth" to /root/.cache/torch/hub/checkpoints/pose_hrne

In [ ]:
# import mmcv
# from mmskeleton.apis import init_pose_estimator, inference_pose_estimator

# cfg = mmcv.Config.fromfile('configs/apis/pose_estimator.cascade_rcnn+hrnet.yaml')
# video = mmcv.VideoReader('resource/data_example/skateboarding.mp4')

# model = init_pose_estimator(**cfg, device=0)
# for i, frame in enumerate(video):
#   result = inference_pose_estimator(model, frame)
#   print('Process the frame {}'.format(i))

KeyboardInterrupt: ignored

### json generation

In [ ]:
import glob
import re
import os.path as osp

classes = [
           "WaveHand",
           "DrinkFromBottle",
           "AnswerPhone",
           "Clapping",
           "TightLace",
           "SitDownStandUp",
           "Stationary",
           "ReadWatch",
           "Bow",
           "ComingIn",
           "GoingOut",
           "G230",
           ]

print("""{
    "categories": [
        "wave",
        "drink from a bottle",
        "answer phone",
        "clap",
        "tight lace",
        "sit down",
        "stand up",
        "read watch",
        "bow",
        "coming in",
        "going out",
        "g230"
    ],
    "annotations": {""")
files = glob.glob("Florence_3d_actions/*.avi")+glob.glob("ccam_actions/*.mp4")
for f in files:
  fname = osp.basename(f)
  if f[0] == 'F':
    ids = list(map(int, re.findall('\d+', fname)))
    cid = ids[3] - 1
  else:
    names = fname.split("_")
    if names[-1] == "comingin.mp4":
      cid = 9
    elif names[-1] == "goingout.mp4":
      cid = 10
    else:
      cid = classes.index(names[0])
  print(f'        "{fname}": {{\n          "category_id": {cid}\n        }},')
print("""\b\b
    }
}""")


### configs/recognition/st_gcn/dataset_fc/train.yaml and test.yml

In [ ]:
!mkdir -p configs/recognition/st_gcn/dataset_fc/

In [ ]:
%%writefile configs/recognition/st_gcn/dataset_fc/train.yaml
argparse_cfg:
  gpus:
    bind_to: processor_cfg.gpus
    help: number of gpus
  work_dir:
    bind_to: processor_cfg.work_dir
    help: the dir to save logs and models
  batch_size:
    bind_to: processor_cfg.batch_size
  resume_from:
    bind_to: processor_cfg.resume_from
    help: the checkpoint file to resume from


processor_cfg:
  type: 'processor.recognition.train'
  workers: 16

  # model setting
  model_cfg:
    type: 'models.backbones.ST_GCN_18'
    in_channels: 3
    num_class: 12
    edge_importance_weighting: True
    graph_cfg:
      layout: 'coco'
      strategy: 'spatial'
  loss_cfg:
    type: 'torch.nn.CrossEntropyLoss'

  # dataset setting
  dataset_cfg:
    # training set
    - type: "datasets.DataPipeline"
      data_source:
        type: "datasets.SkeletonLoader"
        data_dir: ./data/dataset_ccam
        num_track: 2
        num_keypoints: 17
        repeat: 20
      pipeline:
        - {type: "datasets.skeleton.normalize_by_resolution"}
        - {type: "datasets.skeleton.mask_by_visibility"}
        - {type: "datasets.skeleton.pad_zero", size: 150 }
        - {type: "datasets.skeleton.random_crop", size: 150 }
        - {type: "datasets.skeleton.simulate_camera_moving"}
        - {type: "datasets.skeleton.transpose", order: [0, 2, 1, 3]}
        - {type: "datasets.skeleton.to_tuple"}

    - type: "datasets.DataPipeline"
      data_source:
        type: "datasets.SkeletonLoader"
        data_dir: ./data/dataset_ccam
        num_track: 2
        num_keypoints: 17
      pipeline:
        - {type: "datasets.skeleton.normalize_by_resolution"}
        - {type: "datasets.skeleton.mask_by_visibility"}
        - {type: "datasets.skeleton.pad_zero", size: 300 }
        - {type: "datasets.skeleton.random_crop", size: 300 }
        - {type: "datasets.skeleton.transpose", order: [0, 2, 1, 3]}
        - {type: "datasets.skeleton.to_tuple"}

  # dataloader setting
  batch_size: 48
  gpus: 1

  # optimizer setting
  optimizer_cfg:
    type: 'torch.optim.SGD'
    lr: 0.01
    momentum: 0.9
    nesterov: true
    weight_decay: 0.0001

  # runtime setting
  workflow: [['train', 5], ['val', 1]]
  work_dir: ./work_dir/recognition/st_gcn/dataset_florence_ccam
  log_level: 0
  total_epochs: 80
  training_hooks:
    lr_config:
      policy: 'step'
      step: [20, 40, 50, 60]
    log_config:
      interval: 100
      hooks:
        - type: TextLoggerHook
    checkpoint_config:
      interval: 5
    optimizer_config:
      grad_clip:
  resume_from:
  load_from:

Writing configs/recognition/st_gcn/dataset_fc/train.yaml


In [ ]:
%%writefile configs/recognition/st_gcn/dataset_fc/test.yaml
argparse_cfg:
  gpus:
    bind_to: processor_cfg.gpus
    help: number of gpus
  batch_size:
    bind_to: processor_cfg.batch_size
  checkpoint:
    bind_to: processor_cfg.checkpoint
    help: the checkpoint file to load from

processor_cfg:
  type: "processor.recognition.test"
  workers: 16
  #checkpoint: ./work_dir/recognition/st_gcn/dataset_florence_ccam/epoch_10.pth
  checkpoint: ./work_dir/recognition/st_gcn/dataset_florence_ccam/latest.pth

  # model setting
  model_cfg:
    type: "models.backbones.ST_GCN_18"
    in_channels: 3
    num_class: 12
    edge_importance_weighting: True
    graph_cfg:
      layout: "coco"
      strategy: "spatial"

  # dataset setting
  dataset_cfg:
    type: "datasets.DataPipeline"
    data_source:
      type: "datasets.SkeletonLoader"
      data_dir: ./data/dataset_ccam
      num_track: 2
      num_keypoints: 17
    pipeline:
      - { type: "datasets.skeleton.normalize_by_resolution" }
      - { type: "datasets.skeleton.mask_by_visibility" }
      - { type: "datasets.skeleton.pad_zero", size: 300 }
      - { type: "datasets.skeleton.random_crop", size: 300 }
      - { type: "datasets.skeleton.transpose", order: [0, 2, 1, 3] }
      - { type: "datasets.skeleton.to_tuple" }
  # dataloader setting
  batch_size: null
  gpu_batch_size: 64
  gpus: -1


Writing configs/recognition/st_gcn/dataset_fc/test.yaml


## data set exameple test

In [ ]:
#!mmskl configs/utils/build_dataset_example.yaml

In [ ]:
#!mmskl configs/recognition/st_gcn/dataset_example/train.yaml

In [ ]:
#!mmskl configs/recognition/st_gcn/dataset_example/test.yaml

## HRNet and STGCN

### HRNet skeleton data generation

In [ ]:
if False:
  %cd /content/mmskeleton/
  !rm /content/Florence_3d_actions/*.txt
  !rm /content/Florence_3d_actions/.*
  !rm /content/ccam_actions/.*
  !mmskl configs/utils/build_dataset_florence.yaml
  !mmskl configs/utils/build_dataset_ccam.yaml
else:
  !mkdir -p /content/mmskeleton/data/
  %cd /content/mmskeleton/data/
  !unzip -qq /content/dataset_florence.zip
  !unzip -qq /content/dataset_ccam.zip
  %cd /content/mmskeleton/

/content/mmskeleton/data
/content/mmskeleton


### train STGCN model

In [ ]:
!rm -rf ./data/dataset_florence/.ipynb_checkpoints
!mmskl configs/recognition/st_gcn/dataset_fc/train.yaml

INFO:root:Generating grammar tables from /usr/lib/python3.6/lib2to3/Grammar.txt
INFO:root:Generating grammar tables from /usr/lib/python3.6/lib2to3/PatternGrammar.txt
/usr/local/lib/python3.6/dist-packages/Cython/Distutils/old_build_ext.py:41: UserWarning: Cython.Distutils.old_build_ext does not properly handle dependencies and is deprecated.
  "Cython.Distutils.old_build_ext does not properly handle dependencies "
Load configuration information from configs/recognition/st_gcn/dataset_fc/train.yaml
wandb: Currently logged in as: wchoi (use `wandb login --relogin` to force relogin)
2021-02-17 20:14:37.340298: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
wandb: Tracking run with wandb version 0.10.19
wandb: Syncing run sandy-wind-3
wandb: ⭐️ View project at https://wandb.ai/wchoi/STGCN
wandb: 🚀 View run at https://wandb.ai/wchoi/STGCN/runs/1zs7gp37
wandb: Run data is saved locally in /content/mmskeleton/wandb/run-20

### test STGCN model

In [ ]:
!mmskl configs/recognition/st_gcn/dataset_fc/test.yaml

# Extra codes

## Pose estimation using BottomUp method

In [ ]:
%cd /content/mmpose
import os
import re
import cv2
import glob
import pickle
import os.path as osp

### HigherHRNet48

In [ ]:
from mmpose.apis import inference_bottom_up_pose_model, init_pose_model, vis_pose_result

def frame_iter(capture, description = ""):
  def _iterator():
    while capture.grab():
      yield capture.retrieve()

  return tqdm(
    _iterator(),
    desc=description,
    total=int(capture.get(cv2.CAP_PROP_FRAME_COUNT)),
    leave=False,
  )

def inference_pose_estimation_model(video_path, 
                            return_heatmap = False, 
                            save_out_video = True, 
                            out_video_root = '/content/video_results'):
  # build the pose model from a config file and a checkpoint file
  pose_model = init_pose_model(
      '/content/mmpose/configs/bottom_up/higherhrnet/coco/higher_hrnet48_coco_512x512.py', # model configuration
          device='cuda:0')

  dataset = pose_model.cfg.data['test']['type']
  cap = cv2.VideoCapture(video_path)

  if save_out_video:
    os.makedirs(out_video_root, exist_ok=True)
    fps = cap.get(cv2.CAP_PROP_FPS)
    size = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
            int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    videoWriter = cv2.VideoWriter(
        os.path.join(out_video_root,f'vis_{os.path.basename(video_path)}'), fourcc, fps, size)

  # e.g. use ('backbone', ) to return backbone feature
  output_layer_names = ()#('backbone', )
  results = []
  video = frame_iter(cap)
  video.set_postfix({'filename': osp.basename(video_path)})
  for flag, img in video:
    if not flag:
      break

    # inference the model
    pose_results, returned_outputs = inference_bottom_up_pose_model(
      pose_model,
      img,
      return_heatmap=return_heatmap,
      outputs=output_layer_names)
    results.append(pose_results)

    if save_out_video:
      # show the results
      vis_img = vis_pose_result(
        pose_model,
        img,
        pose_results,
        dataset=dataset,
        kpt_score_thr=0.3,
        show=False)
      
      videoWriter.write(vis_img)

  video.close()
  cap.release()
  if save_out_video:
      videoWriter.release()
  cv2.destroyAllWindows()
  
  return results

### Run the inference on the Florence 3D action dataset.

In [ ]:
save_out_video = False

video_list = glob.glob('/content/Florence_3d_actions/*.avi')
video_list.sort(reverse=True)
pose_estimation_results = {}
with tqdm(total=len(video_list)) as pbar:
  for video_path in video_list:
    filename = osp.basename(video_path)
    results = inference_pose_estimation_model(video_path, save_out_video = save_out_video)
    pose_estimation_results[filename] = results
    pbar.update(1)
    if save_out_video:
      video_result = video_path.replace('Florence_3d_actions/','video_results/vis_')
      video_result1 = video_result.replace('avi','mp4')
      !ffmpeg -y -loglevel panic -i $video_result -vcodec libx264 $video_result1
      !rm $video_result

In [ ]:
save_out_video = False

video_list = glob.glob('/content/ccam_actions/*.mp4')
video_list.sort(reverse=True)
with tqdm(total=len(video_list)) as pbar:
  for video_path in video_list:
    filename = osp.basename(video_path)
    results = inference_pose_estimation_model(video_path, save_out_video = save_out_video)
    pose_estimation_results[filename] = results
    pbar.update(1)
    if save_out_video:
      video_result = video_path.replace('Florence_3d_actions/','video_results/vis_')
      video_result1 = video_result.replace('avi','mp4')
      !ffmpeg -y -loglevel panic -i $video_result -vcodec libx264 $video_result1
      !rm $video_result

In [ ]:
import random
video_result_list = glob.glob('/content/video_results/*.mp4')
video_result = random.choice(video_result_list)
show_local_mp4_video(video_result)

In [ ]:
print(pose_estimation_results.keys())
pickle.dump(pose_estimation_results, open("/content/pose_estimation_results_higherhrnet48.pkl", "wb" )) # down

In [ ]:
# Change current working directory to /content
%cd /content
!zip -q -r video_results_higherhrnet48.zip video_results

## Action Recognition (simple ST-GCN)


In [ ]:
%%shell
cd /content
rm -rf st-gcn-pytorch
git clone https://github.com/taznux/st-gcn-pytorch
cd st-gcn-pytorch
mkdir dataset models
ln -sf /content/Florence_3d_actions dataset/

### STGCN Human Action Recognition on Florence 3D skeletion data

In [ ]:
%cd st-gcn-pytorch
#!python preprocess.py # skeleton conversion

In [ ]:
import random
import pickle
import re
import torch
import numpy as np

data = pose_estimation_results

train = []
valid = []
test  = []
train_label = []
valid_label = []
test_label  = []
classes = [
           "WaveHand",
           "DrinkFromBottle",
           "AnswerPhone",
           "Clapping",
           "TightLace",
           "SitDownStandUp",
           "Stationary",
           "ReadWatch",
           "Bow",
           "G230"
           ]

vid = 0
for d in data:
  ids = list(map(int, re.findall('\d+', d)))
  try:
    gid, tid, aid, cid = ids
  except:
    class_name = d.split("_")[0]
    gid, tid, aid, cid = 11,11,11,classes.index(class_name)+1

  print(vid, d)
  frames = []
  for frame in data[d]:
    if len(frame) > 0:
      pose = frame[0]
      keypoints = np.array(pose['keypoints'])[:17]
      keypoints[:,2] = 0.5
      frames.append(keypoints)
  
  if len(frames) >= 32:
    frames = random.sample(frames, 32)
    frames = torch.from_numpy(np.stack(frames, 0))
  else:
    frames = np.stack(frames, 0)
    xloc = np.arange(frames.shape[0])
    new_xloc = np.linspace(0, frames.shape[0], 32)
    frames = np.reshape(frames, (frames.shape[0], -1)).transpose()

    new_datas = []
    for fr in frames:
      new_datas.append(np.interp(new_xloc, xloc, fr))
    frames = torch.from_numpy(np.stack(new_datas, 0)).t()

  frames = frames.view(32, -1, 3)

  if tid < 9:
    train.append(frames)
    train_label.append(cid-1)
  elif tid < 10:
    valid.append(frames)
    valid_label.append(cid-1)
  else:
    test.append(frames)
    test_label.append(cid-1)

  vid += 1
  
train_label = torch.from_numpy(np.asarray(train_label))
valid_label = torch.from_numpy(np.asarray(valid_label))
test_label  = torch.from_numpy(np.asarray(test_label))

torch.save((torch.stack(train, 0), train_label), '/content/st-gcn-pytorch/dataset/train.pkl')
torch.save((torch.stack(valid, 0), valid_label), '/content/st-gcn-pytorch/dataset/valid.pkl')
torch.save((torch.stack(test, 0),  test_label),  '/content/st-gcn-pytorch/dataset/test.pkl')

In [ ]:
%cd /content/st-gcn-pytorch
#!python preprocess.py # skeleton conversion
!python main.py --num_epochs 100 --batch_size 1 --val_step 1 --dropout_rate 0.05 --weight_decay 0.001 # train and test

In [ ]:
!python main.py --mode test --test_epoch 21

In [ ]:
import os
import numpy as np
import torch
import torch.optim as optim
import torch.utils.data as data

from model import *
from metric import accuracy
from config import get_args
args = get_args()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_tensor, train_label = torch.load(args.train_path)
valid_tensor, valid_label = torch.load(args.valid_path)
test_tensor, test_label = torch.load(args.test_path)
train_tensor = train_tensor.to(device)
valid_tensor = valid_tensor.to(device)
test_tensor = test_tensor.to(device)
train_label = train_label.to(device)
valid_label = valid_label.to(device)
test_label = test_label.to(device)
train_loader = data.DataLoader(data.TensorDataset(train_tensor, train_label),
                               batch_size=args.batch_size, shuffle=False)
valid_loader = data.DataLoader(data.TensorDataset(valid_tensor, valid_label),
                               batch_size=args.batch_size, shuffle=False)
test_loader = data.DataLoader(data.TensorDataset(test_tensor, test_label),
                              batch_size=args.batch_size, shuffle=False)


'''
"keypoints": {
    0: "nose",
    1: "left_eye",
    2: "right_eye",
    3: "left_ear",
    4: "right_ear",
    5: "left_shoulder",
    6: "right_shoulder",
    7: "left_elbow",
    8: "right_elbow",
    9: "left_wrist",
    10: "right_wrist",
    11: "left_hip",
    12: "right_hip",
    13: "left_knee",
    14: "right_knee",
    15: "left_ankle",
    16: "right_ankle"
},
"skeleton": [
    [16,14],[14,12],[17,15],[15,13],[12,13],[6,12],[7,13], [6,7],[6,8],
    [7,9],[8,10],[9,11],[2,3],[1,2],[1,3],[2,4],[3,5],[4,6],[5,7]]
'''
A = [[0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
     [1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
     [1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
     [0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
     [0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
     [0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0],
     [0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0],
     [0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
     [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
     [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
     [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
     [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0],
     [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0],
     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0],
     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1],
     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]]
A = torch.from_numpy(np.asarray(A)).to(device)

model = GGCN(A, train_tensor.size(3), args.num_classes,
             [train_tensor.size(3), train_tensor.size(
                 3)*3], [train_tensor.size(3)*3, 16, 32, 64],
             args.feat_dims, args.dropout_rate)
if device == 'cuda':
    model.cuda()

num_params = 0
for p in model.parameters():
    num_params += p.numel()
print(model)
print('The number of parameters: {}'.format(num_params))

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=args.learning_rate,
                       betas=[args.beta1, args.beta2], weight_decay=args.weight_decay)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)

best_epoch = 0
best_epoch1 = 0
best_acc = 0
best_loss = 10


def train():
    global best_epoch, best_epoch1, best_acc, best_loss

    if args.start_epoch:
        model.load_state_dict(torch.load(os.path.join(args.model_path,
                                                      'model-%d.pkl' % (args.start_epoch))))

    # Training
    for epoch in range(args.start_epoch, args.num_epochs):
        train_loss = 0
        train_acc = 0

        model.train()
        for i, (x, target) in enumerate(train_loader, start=1):
            logit = model(x.float())

            loss = criterion(logit, target)

            model.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            train_acc += accuracy(logit, target)
        scheduler.step()

        print('[epoch', epoch+1, '] Train loss:',
              train_loss/i, 'Train Acc:', train_acc/i)

        if (epoch+1) % args.val_step == 0:
            model.eval()
            val_loss = 0
            val_acc = 0
            with torch.no_grad():
                for i, (x, target) in enumerate(valid_loader, start=1):
                    logit = model(x.float())

                    val_loss += criterion(logit, target).item()
                    val_acc += accuracy(logit, target)

                if best_acc < (val_acc/i):
                    best_epoch = epoch+1
                    best_acc = (val_acc/i)
                    torch.save(model.state_dict(), os.path.join(
                        args.model_path, 'model-%d.pkl' % (best_epoch)))
                if best_loss > (val_loss/i):
                    best_epoch1 = epoch+1
                    best_loss = (val_loss/i)
                    torch.save(model.state_dict(), os.path.join(
                        args.model_path, 'model-%d.pkl' % (best_epoch1)))

            print('Val loss:', val_loss/i, 'Val Acc:', val_acc/i)
    print(f'best_epoch: {best_epoch}, {best_epoch1}')


def test():
    global best_epoch

    model.load_state_dict(torch.load(os.path.join(args.model_path,
                                                  'model-%d.pkl' % (best_epoch))))
    print("load model from 'model-%d.pkl'" % (best_epoch))

    model.eval()
    test_loss = 0
    test_acc = 0
    with torch.no_grad():
        for i, (x, target) in enumerate(test_loader, start=1):
            logit = model(x.float())
            print(i, target.cpu()[0].numpy(), torch.max(logit, 1)[1].float().cpu()[0].numpy())

            try:
              if target < 9:
                test_loss += criterion(logit, target).item()
                test_acc += accuracy(logit, target)
            except:
              continue

    print('Test loss:', test_loss/i, 'Test Acc:', test_acc/i)


if __name__ == '__main__':
    if args.mode == 'train':
        train()
    elif args.mode == 'test':
        best_epoch = args.test_epoch
    test()
